<a href="https://colab.research.google.com/github/Piyush47200/Supply-chain-analytics/blob/main/Supplier_chain_network_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Limitation: Only 1 port.

Install libraries

In [577]:
!pip install geopy
!pip install openrouteservice
!pip install gurobipy

Import libraries

In [578]:
from geopy.geocoders import Nominatim
import pandas as pd
import time
import openrouteservice
import math

Load port, DC and customer sheets.

In [579]:
df_port_city=pd.read_excel('Port.xlsx')
df_DC_city=pd.read_excel('DC.xlsx')
df_customer_city=pd.read_excel('Customer.xlsx')

Defining dataframes for port, DC and customer to hold coordinates data.

In [580]:
df_coordinate_port_city=pd.DataFrame(columns=['City','Latitude','Longitude'])
df_coordinate_DC_city=pd.DataFrame(columns=['City','Latitude','Longitude'])
df_coordinate_customer_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude', 'Demand'])

Finding coordinates for ports

In [581]:
geolocator = Nominatim(user_agent="your_email_or_app_name")
for i in range(0, len(df_port_city)):
  location = geolocator.geocode(df_port_city['City'].iloc[i])
  if location:
    df_coordinate_port_city.loc[i] = [df_port_city['City'].iloc[i], location.latitude, location.longitude]
  else:
    df_coordinate_port_city.loc[i] = [df_port_city['City'].iloc[i], None, None]

Finding coordinates for DC

In [582]:
import pandas as pd
from geopy.geocoders import Nominatim

# Assuming df_coordinate_DC_city was initialized as:
# df_coordinate_DC_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude'])

# Modify the code as follows:
# Define columns for df_coordinate_DC_city if it doesn't exist
if df_coordinate_DC_city.empty:
  df_coordinate_DC_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude', 'Fixed Cost'])

for i in range(0, len(df_DC_city)):
  location = geolocator.geocode(df_DC_city['City'].iloc[i])
  if location:
    # Assign values to the row, including the 'Fixed Cost' column
    df_coordinate_DC_city.loc[i] = [df_DC_city['City'].iloc[i], location.latitude, location.longitude, df_DC_city['Fixed Cost'].iloc[i]]
  else:
    # Assign values to the row, including the 'Fixed Cost' column and None for coordinates
    df_coordinate_DC_city.loc[i] = [df_DC_city['City'].iloc[i], None, None, df_DC_city['Fixed Cost'].iloc[i]]

Finding coordinates for customer

In [583]:
geolocator = Nominatim(user_agent="your_email_or_app_name")
# Increased timeout to 10 seconds
for i in range(0, len(df_customer_city)):
  try:
    location = geolocator.geocode(df_customer_city['City'].iloc[i], timeout=10)
    if location:
      df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], location.latitude, location.longitude,df_customer_city['Demand'].iloc[i]]
    else:
      df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
    #time.sleep(2) # Wait 2 seconds between requests to avoid rate limiting
  except GeocoderTimedOut:
    print("Geocoding timed out for city:", df_customer_city['City'].iloc[i])
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
  except GeocoderUnavailable:
    print("Geocoder service unavailable for city:", df_customer_city['City'].iloc[i])
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
  except Exception as e:
    print("Error geocoding city:", df_customer_city['City'].iloc[i], "-", e)
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]

Exporting coordintates of customers to excel.

In [584]:
df_coordinate_customer_city.to_excel('coordinate_customer_city.xlsx', index=False)

In [585]:
print(df_coordinate_customer_city)

          City   Latitude  Longitude  Demand
0    Bengaluru  12.976794  77.590082    3000
1      Madurai   9.926115  78.114098    3200
2        Kochi   9.967428  76.245444    3300
3      Chennai  13.083694  80.270186    3400
4       Mysuru  12.305183  76.655361    3500
5       Meerut  28.996330  77.706192    3600
6      Aligarh  27.876107  78.135815    3700
7         Agra  27.175255  78.009816    3800
8   Chandigarh  30.729844  76.784146    3900
9      Bhiwani  28.793170  76.139128    4000
10      Jaipur  26.915458  75.818982    4100
11      Kanpur  26.460914  80.321759    4200


In [586]:
print(df_coordinate_port_city)

          City   Latitude  Longitude
0  NHAVA SHEVA  18.945031  72.949423


In [587]:
print(df_coordinate_DC_city)

         City   Latitude  Longitude  Fixed Cost
0   New Delhi  28.613895  77.209006       10000
1  Coimbatore  11.001812  76.962842       20000
2     Kolkata  22.572646  88.363895     2000000
3      Mumbai  19.054999  72.869203     2000000


Defining two new dataframes for distance calculations

In [588]:
df_distance_port_DC=pd.DataFrame(columns=['Source','Destination','Distance'])
df_distance_DC_customer=pd.DataFrame(columns=['Source','Destination','Distance'])

Defining a function which will calculate the distance

In [589]:
def distance(lat1, lon1, lat2, lon2):
  client = openrouteservice.Client(key='5b3ce3597851110001cf62489d683e3d831942f3adab13e6a2e5a7dc')

# Coordinates of New Delhi and Kolkata (longitude, latitude)
  source = (lon1, lat1)
  destination = (lon2, lat2)
  routes = client.directions(coordinates=[source, destination],profile='driving-hgv', format='geojson')

  distance_meters = routes['features'][0]['properties']['segments'][0]['distance']
  distance_km = distance_meters / 1000
  #print(f"Road distance between New Delhi and Kolkata is {distance_km:.2f} km")
  return distance_km

Estimating distance for port to DC

In [590]:
k=0
for j in range(0,len(df_coordinate_port_city)):
  for i in range(0,len(df_coordinate_DC_city)):
    dist=distance(df_coordinate_port_city['Latitude'].iloc[j], df_coordinate_port_city['Longitude'].iloc[j], df_coordinate_DC_city['Latitude'].iloc[i], df_coordinate_DC_city['Longitude'].iloc[i])
    df_distance_port_DC.loc[k]=[df_coordinate_port_city['City'].iloc[j],df_coordinate_DC_city['City'].iloc[i],dist]
    k=k+1


In [591]:
print(df_distance_port_DC)

        Source Destination   Distance
0  NHAVA SHEVA   New Delhi  1482.7279
1  NHAVA SHEVA  Coimbatore  1324.2070
2  NHAVA SHEVA     Kolkata  1932.8627
3  NHAVA SHEVA      Mumbai    47.8468


Estimating distance for DC to customer

In [592]:
k=0
for j in range(0,len(df_coordinate_DC_city)):
  for i in range(0,len(df_coordinate_customer_city)):
    dist=distance(df_coordinate_DC_city['Latitude'].iloc[j], df_coordinate_DC_city['Longitude'].iloc[j], df_coordinate_customer_city['Latitude'].iloc[i], df_coordinate_customer_city['Longitude'].iloc[i])
    df_distance_DC_customer.loc[k]=[df_coordinate_DC_city['City'].iloc[j],df_coordinate_customer_city['City'].iloc[i],dist]
    k=k+1

/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rat

In [593]:
print(df_distance_DC_customer)

        Source Destination   Distance
0    New Delhi   Bengaluru  2110.0153
1    New Delhi     Madurai  2542.9761
2    New Delhi       Kochi  2650.2621
3    New Delhi     Chennai  2173.4354
4    New Delhi      Mysuru  2245.2512
5    New Delhi      Meerut    83.1616
6    New Delhi     Aligarh   157.6321
7    New Delhi        Agra   200.9726
8    New Delhi  Chandigarh   258.5782
9    New Delhi     Bhiwani   125.2304
10   New Delhi      Jaipur   272.6346
11   New Delhi      Kanpur   450.0073
12  Coimbatore   Bengaluru   338.9688
13  Coimbatore     Madurai   228.0159
14  Coimbatore       Kochi   197.8971
15  Coimbatore     Chennai   513.7791
16  Coimbatore      Mysuru   208.4256
17  Coimbatore      Meerut  2500.2858
18  Coimbatore     Aligarh  2355.9328
19  Coimbatore        Agra  2264.1069
20  Coimbatore  Chandigarh  2772.9097
21  Coimbatore     Bhiwani  2508.8106
22  Coimbatore      Jaipur  2259.6561
23  Coimbatore      Kanpur  2143.8341
24     Kolkata   Bengaluru  1887.1274
25     Kolka

In [594]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("MyModel")

In [595]:
# Create an environment with your WLS license
params = {
"WLSACCESSID": '803f8182-ebee-4591-8370-b64a82f775ea',
"WLSSECRET": '7696ca12-c44b-46e1-9875-a948ff7c3fdf',
"LICENSEID": 2574149,
}
env = gp.Env(params=params)
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2574149
Academic license 2574149 - for non-commercial use only - registered to pi___@gmail.com


Defining known variables

In [596]:
c = 0.1
r = 0
for i in range(0, len(df_customer_city)):
  r = r + df_customer_city['Demand'][i]

Defining unknown variables

In [597]:
x_var = {}
y_var = {}
z_var = {}

In [598]:
z = 0
for i in range(0, len(df_DC_city)):
  z_var[f"z[{i}]"] = model.addVar(vtype=GRB.BINARY, name=f"z[{i}]")
  z = z + z_var[f"z[{i}]"]

In [599]:
x = 0
for j in range(0, len(df_DC_city)):
  for i in range(0, len(df_port_city)):
    x_var[f"x[{i+j}]"] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"x[{i+j}]")
    x = x + x_var[f"x[{i+j}]"]*z_var[f"z[{j}]"]

In [600]:
y = 0
for j in range(0, len(df_DC_city)):
  for i in range(0, len(df_DC_city)*len(df_customer_city)):
    y_var[f"y[{i}]"] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"y[{i}]")
    y = y + y_var[f"y[{i}]"]*z_var[f"z[{j}]"]

Constraints

Demand of each customer is exactly met from DCs.

In [601]:
for j in range(0, len(df_customer_city)):
  a = 0
  for i in range(0, len(df_DC_city)):
    a = a + y_var[f"y[{j+len(df_customer_city)*i}]"]
  model.addConstr(a == df_customer_city['Demand'][j])

1. Outgoing from ports = total demand
2. Outgoing from DC = total demand
3. atleast one DC is opened

In [602]:
model.addConstr(x == r, "Demand_x")
model.addConstr(y == r, "Demand_y")
model.addConstr(z >= 1, "Z_non_zero")

<gurobi.Constr *Awaiting Model Update*>

Total incoming at a DC = Total outgoing from a DC

In [603]:
incoming = 0
outgoing = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    incoming = incoming + x_var[f"x[{i+len(df_DC_city)*j}]"]
  for k in range(0, len(df_customer_city)):
    outgoing = outgoing + y_var[f"y[{k+len(df_customer_city)*i}]"]
  model.addConstr(incoming == outgoing)

Pre-work for objective function

In [604]:
p = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    p = p + df_distance_port_DC['Distance'][i+len(df_DC_city)*j]*x_var[f"x[{i+len(df_DC_city)*j}]"]*z_var[f"z[{i}]"]
p = p*c #This multiplication is fine as X is the linear expression before z is applied

In [605]:
q = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_customer_city)):
    q = q + df_distance_DC_customer['Distance'][len(df_customer_city)*i+j]*y_var[f"y[{len(df_customer_city)*i+j}]"]*z_var[f"z[{i}]"]
q = q*c

In [606]:
s = 0
for i in range(0, len(df_DC_city)):
  s = s + df_DC_city['Fixed Cost'][i]*z_var[f"z[{i}]"]

Objective function

In [607]:
objective =  p + q + s

In [608]:
model.setObjective(objective, GRB.MINIMIZE)

In [609]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2574149 - for non-commercial use only - registered to pi___@gmail.com
Optimize a model with 17 rows, 200 columns and 182 nonzeros
Model fingerprint: 0x24540545
Model has 52 quadratic objective terms
Model has 2 quadratic constraints
Variable types: 0 continuous, 200 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+04, 2e+06]
  QObjective range [1e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
  QRHS range       [4e+04, 4e+04]
Presolve time: 0.00s
Presolved: 483 rows, 896 columns, 1451 nonzeros
Presolved model has 464 SOS constraint(s)
Variable types: 0 continuous, 896 integer (236 binary)
Found heuristic solution: objectiv

In [610]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.ObjVal}")
else:
    print("No optimal solution found.")

Optimal objective value: 7353494.822999999


In [611]:
for i in range(0, len(df_DC_city)):
  print(z_var[f"z[{i}]"])

<gurobi.Var z[0] (value 1.0)>
<gurobi.Var z[1] (value 1.0)>
<gurobi.Var z[2] (value 0.0)>
<gurobi.Var z[3] (value 0.0)>


Visualization

In [612]:
!pip install folium
import folium

In [613]:
lat = 0
long = 0
n = 0
for i in range(0, len(df_coordinate_DC_city)):
  lat = lat + df_coordinate_DC_city['Latitude'][i]*z_var[f"z[{i}]"].x
  long = long + df_coordinate_DC_city['Longitude'][i]*z_var[f"z[{i}]"].x
  n = n + z_var[f"z[{i}]"].x
lat = lat/n
long = long/n
m = folium.Map(location=[lat, long], zoom_start=5.1)

In [614]:
for i in range(0, len(df_port_city)):
    # Providing location as a list [latitude, longitude]
    folium.Marker(
        location=[df_coordinate_port_city['Latitude'][i], df_coordinate_port_city['Longitude'][i]],
        popup=f"{df_coordinate_port_city['City'][i]}",
        icon=folium.Icon(color="darkgreen", icon="ship", prefix="fa")
    ).add_to(m)

In [615]:
for i in range(0, len(df_customer_city)):
    # Providing location as a list [latitude, longitude]
    folium.Marker(
        location=[df_coordinate_customer_city['Latitude'][i], df_coordinate_customer_city['Longitude'][i]],
        popup=f"{df_coordinate_customer_city['City'][i]}",
        icon=folium.Icon(color="red", icon="user", prefix="fa")
    ).add_to(m)

In [616]:
for i in range(0, len(df_DC_city)):
    # Providing location as a list [latitude, longitude]
    if z_var[f"z[{i}]"].x == 1:
      folium.Marker(location=[df_coordinate_DC_city['Latitude'][i], df_coordinate_DC_city['Longitude'][i]], popup=f"{df_coordinate_DC_city['City'][i]}",
        icon=folium.Icon(color="black", icon="warehouse", prefix="fa")).add_to(m)

In [617]:
for i in range(0, len(df_port_city)*len(df_DC_city)):
  if z_var[f"z[{i}]"].x == 1:
    sourc_city = df_distance_port_DC['Source'][i]
    destination_city = df_distance_port_DC['Destination'][i]
    source_lat = df_coordinate_port_city.loc[df_coordinate_port_city['City'] == sourc_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    source_lon = df_coordinate_port_city.loc[df_coordinate_port_city['City'] == sourc_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]
    destination_lat = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == destination_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    destination_lon = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == destination_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]

    folium.PolyLine(
        locations=[(source_lat, source_lon), (destination_lat, destination_lon)],
        color="darkgreen",
    ).add_to(m)

In [618]:
for i in range(0, len(df_DC_city)*len(df_customer_city)):
  if y_var[f"y[{i}]"].x > 1:
    # Get the source and destination cities
    source_city = df_distance_DC_customer['Source'][i]
    destination_city = df_distance_DC_customer['Destination'][i]

    # Get the coordinates for the source and destination cities
    source_lat = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == source_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    source_lon = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == source_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]
    destination_lat = df_coordinate_customer_city.loc[df_coordinate_customer_city['City'] == destination_city, 'Latitude'].iloc[0] #Access the first element using iloc[0]
    destination_lon = df_coordinate_customer_city.loc[df_coordinate_customer_city['City'] == destination_city, 'Longitude'].iloc[0] #Access the first element using iloc[0]

    # Add the polyline to the map
    folium.PolyLine(
        locations=[(source_lat, source_lon), (destination_lat, destination_lon)],
        color="orange",
    ).add_to(m)

In [619]:
m.save("my_map.html")

In [620]:
m